In [11]:
import torch
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import torch.nn as nn
import pickle
import random as rd
import torch.optim as optim
import numpy as np
import sys as s
import matplotlib.pyplot as plt
import math


#           GERANDO O BANCO DE DADOS

In [12]:
from Dataset_Generator import Dataset

In [13]:
Dataset(2,2,300)

XC = (2, 2, 1)
inp = torch.Size([2, 2, 300])
out = torch.Size([2, 2, 300])


In [14]:
inp = pickle.load(open("inp", "rb"))
out = pickle.load(open("out", "rb"))
XC = pickle.load(open("XC", "rb"))
n_batch = np.shape(inp)[0]
batch_size = np.shape(inp)[1]
n_examples = np.shape(inp)[2]

In [15]:
print(inp[0][0])
print(out[0][0])

tensor([  0.0000,   1.0033,   2.0067,   3.0100,   4.0134,   5.0167,   6.0201,
          7.0234,   8.0268,   9.0301,  10.0334,  11.0368,  12.0401,  13.0435,
         14.0468,  15.0502,  16.0535,  17.0569,  18.0602,  19.0635,  20.0669,
         21.0702,  22.0736,  23.0769,  24.0803,  25.0836,  26.0870,  27.0903,
         28.0936,  29.0970,  30.1003,  31.1037,  32.1070,  33.1104,  34.1137,
         35.1171,  36.1204,  37.1237,  38.1271,  39.1304,  40.1338,  41.1371,
         42.1405,  43.1438,  44.1472,  45.1505,  46.1538,  47.1572,  48.1605,
         49.1639,  50.1672,  51.1706,  52.1739,  53.1773,  54.1806,  55.1839,
         56.1873,  57.1906,  58.1940,  59.1973,  60.2007,  61.2040,  62.2074,
         63.2107,  64.2140,  65.2174,  66.2207,  67.2241,  68.2274,  69.2308,
         70.2341,  71.2375,  72.2408,  73.2441,  74.2475,  75.2508,  76.2542,
         77.2575,  78.2609,  79.2642,  80.2676,  81.2709,  82.2742,  83.2776,
         84.2809,  85.2843,  86.2876,  87.2910,  88.2943,  89.29

# 
# DEFINE O MODELO
# 



In [16]:
class Autoencoder(nn.Module):
    def __init__(self):
        # N, 50
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(1, 300),
            nn.ELU(),
            nn.Linear(300, 600),
            nn.ELU(),
            nn.Linear(600, 300),
            nn.ELU(),
            nn.Linear(300, 1),
            nn.ELU(),
        )        
    def forward(self, x):
        encoded = self.encoder(x)
        return encoded


# -------------------------------------------------------------------------------
# ------------------CHAMA O MODELO E INICIA CAMADAS DE PESOS ORTOGONAIS----------
# -------------------------------------------------------------------------------
model = Autoencoder()
for m in model.modules():
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.orthogonal_(m.weight)
criterion = nn.MSELoss()  # segundo a investigar
# ,lr=1e-4,weight_decay = 1e-5)
optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.SGD(model.parameters(),lr=1e-4,weight_decay = 1e-5)#,momentum=0.5)
# -------------------------------------------------------------------------------



# ------TREINO DO DECODER MODIFICADO--O[5]+Q[11] >> OUTPUT[1]--------------------
# -------------------------------------------------------------------------------

In [17]:
def treine(epochs):
    inp = pickle.load(open("inp", "rb"))
    out = pickle.load(open("out", "rb"))
    n_batch = np.shape(inp)[0]
    batch_size = np.shape(inp)[1]
    n_examples = np.shape(inp)[2]
    for epoch in range(epochs):
        for batch_idx in range(n_batch):
            O = inp[batch_idx]
            A = out[batch_idx]
            O = O.float()
            A = A.float()
            for i in O:
                recon, latent = model(i)
                loss = torch.mean((recon-A)**2)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        print(f'Epoch:{epoch+1},Loss:{loss.item():.4f}')



treine(200)
# print('end')
# -------------------------------------------------------------------------------
# --------------------- SALVANDO-------------------------------------------------
# -------------------------------------------------------------------------------
# PATH_save='Estado_Box_2_with_two_latent.pt'
##PATH_load = 'Estado_Box_2_with_two_latent.pt'
#torch.save(model.state_dict(), PATH_save)
# s.exit()
##model.load_state_dict(torch.load(PATH_load))
#torch.save(model.state_dict(), PATH_save)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x300 and 1x300)

# 
# GRÁFICOS
# 

In [ ]:
def Latent_values_Scynet():
    for aux in range(n_batch):
        O = inp[aux].float()
        Q = question[aux].float()
        A = out[aux].float()
        j = J[aux]
        x = np.zeros(np.shape(j)[0])
        y1 = np.zeros(np.shape(j)[0])
        y2 = np.zeros(np.shape(j)[0])
        recon, latent = model(O, Q)
        for i in range(0, 499):
            x[i] = j[i]
            y1[i] = latent[i, 0]
            y2[i] = latent[i, 1]
        plt.scatter(x, y1, label='Latent Activation 1')
        plt.scatter(x, y2, label='Latent Activation 2')
        plt.xlabel('Momento angular total')
        plt.ylabel('Latent Activation')
        plt.legend()
        plt.pause(1.5)
        plt.close()

    plt.show()


Latent_values_Scynet()
# s.exit()
# -------------------------------------------------------------------------------
